In [ ]:
!kaggle datasets download -d uciml/pima-indians-diabetes-database


Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 15.5MB/s]


In [ ]:
# prompt: !kaggle datasets download -d uciml/pima-indians-diabetes-database

!unzip pima-indians-diabetes-database.zip


Archive:  pima-indians-diabetes-database.zip
  inflating: diabetes.csv            


In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [ ]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [ ]:
X.shape

(768, 8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
# model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [ ]:
history = model.fit(batch_size=32, x=X_train, y=y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5243 - loss: 0.7070 - val_accuracy: 0.5779 - val_loss: 0.6843
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6563 - loss: 0.6462 - val_accuracy: 0.6299 - val_loss: 0.6415
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6914 - loss: 0.5880 - val_accuracy: 0.6494 - val_loss: 0.6111
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7186 - loss: 0.5521 - val_accuracy: 0.6883 - val_loss: 0.5906
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7427 - loss: 0.5333 - val_accuracy: 0.6948 - val_loss: 0.5733
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7526 - loss: 0.5129 - val_accuracy: 0.6883 - val_loss: 0.5602
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7608 - loss: 0.4951 - val_accuracy: 0.6753 - val_loss: 0.5497
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7455 - loss: 0.4990 - val_accuracy: 0.6883 -

In [ ]:
# 1. how to select appropriate optimizers
# 2. No. of nodes in a layer
# 3. how to select no. of layers
# 4. All in one model

In [ ]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
def buildmodel(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))
  optimizers = hp.Choice('optimizer', values =  ['adam', 'sgd', 'rmsprop','adadelta'])
  model.compile(optimizer = optimizers , loss = 'binary_crossentropy' , metrics = ['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(buildmodel, objective='val_accuracy', max_trials=5)

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 09s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 24s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,batch_size = 32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.7625 - loss: 0.5132 - val_accuracy: 0.7143 - val_loss: 0.5223
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7583 - loss: 0.4936 - val_accuracy: 0.7143 - val_loss: 0.5189
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7529 - loss: 0.4852 - val_accuracy: 0.7273 - val_loss: 0.5143
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7447 - loss: 0.4848 - val_accuracy: 0.7143 - val_loss: 0.5117
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7709 - loss: 0.4711 - val_accuracy: 0.7273 - val_loss: 0.5092
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7607 - loss: 0.4794 - val_accuracy: 0.7208 - val_loss: 0.5091
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7714 - loss: 0.4622 - val_accuracy: 0.7273 - val_loss: 0.5085
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7544 - loss: 0.4794 - val_accurac

In [ ]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value=8, max_value=128, step=8)
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1,activation = 'sigmoid'))
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5,directory='kt_dir',project_name = 'homelander')

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 14s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,batch_size = 32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7735 - loss: 0.4709 - val_accuracy: 0.7792 - val_loss: 0.5017
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7467 - loss: 0.4898 - val_accuracy: 0.7727 - val_loss: 0.5044
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7814 - loss: 0.4574 - val_accuracy: 0.7727 - val_loss: 0.5042
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7752 - loss: 0.4686 - val_accuracy: 0.7662 - val_loss: 0.5029
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7938 - loss: 0.4319 - val_accuracy: 0.7597 - val_loss: 0.5043
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8148 - loss: 0.4114 - val_accuracy: 0.7662 - val_loss: 0.5076
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7917 - loss: 0.4289 - val_accuracy: 0.7662 - val_loss: 0.5116
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7787 - loss: 0.4270 - val_accuracy: 0.76

In [ ]:
def built_model(hp):
  model = Sequential()

  model.add(Dense(128, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(128, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(built_model, objective='val_accuracy', max_trials=5,directory='kt_dir',project_name = 'bucher')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 28s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,batch_size = 32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.7793 - loss: 0.4697 - val_accuracy: 0.7597 - val_loss: 0.5166
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7941 - loss: 0.4334 - val_accuracy: 0.7532 - val_loss: 0.5281
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7951 - loss: 0.4098 - val_accuracy: 0.7403 - val_loss: 0.5363
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8136 - loss: 0.3819 - val_accuracy: 0.7143 - val_loss: 0.5561
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8202 - loss: 0.3904 - val_accuracy: 0.6688 - val_loss: 0.6161
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8438 - loss: 0.3794 - val_accuracy: 0.7013 - val_loss: 0.5947
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8236 - loss: 0.3632 - val_accuracy: 0.7532 - val_loss: 0.6395
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8512 - loss: 0.3222 - val_accuracy: 0.70

In [ ]:
# Now lets build a final function with all the best paramaters to be selected by tuner

In [ ]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8), activation='relu', input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8), activation='relu'))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9])))
    counter += 1
  model.add(Dense(1, activation='sigmoid'))
  optimizer = hp.Choice('optimizer', values =  ['adam', 'sgd', 'rmsprop','adadelta','nadam'])
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=3,directory='mydir',project_name = 'Final')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 18s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 80,
 'dropout0': 0.1,
 'optimizer': 'rmsprop',
 'units1': 24,
 'dropout1': 0.3,
 'units2': 120,
 'dropout2': 0.5,
 'units3': 80,
 'dropout3': 0.4,
 'units4': 16,
 'dropout4': 0.4,
 'units5': 8,
 'dropout5': 0.1,
 'units6': 16,
 'dropout6': 0.8,
 'units7': 16,
 'dropout7': 0.4,
 'units8': 104,
 'dropout8': 0.1,
 'units9': 88,
 'dropout9': 0.8}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7682 - loss: 0.4885 - val_accuracy: 0.7532 - val_loss: 0.5205
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7601 - loss: 0.4763 - val_accuracy: 0.7403 - val_loss: 0.5233
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7935 - loss: 0.4200 - val_accuracy: 0.7403 - val_loss: 0.5262
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7529 - loss: 0.4809 - val_accuracy: 0.7597 - val_loss: 0.5232
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7761 - loss: 0.4571 - val_accuracy: 0.7468 - val_loss: 0.5204
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7746 - loss: 0.4512 - val_accuracy: 0.7532 - val_loss: 0.5220
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7839 - loss: 0.4464 - val_accuracy: 0.7532 - val_loss: 0.5230
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7804 - loss: 0.4481 - val_accuracy: 0.75

In [ ]:
model.evaluate(X_test,y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7550 - loss: 0.6698 


[0.7305036783218384, 0.7337662577629089]